# Titanic

# Libraries

In [24]:
import pandas as pd

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

import tensorflow as tf
from keras.layers import Dense
from keras import Sequential
from keras.layers import Dropout
import numpy as np
import matplotlib.pyplot as plt

# Data

In [25]:
df_train = pd.read_csv('./datasets/train.csv', index_col='PassengerId')
df_test = pd.read_csv('./datasets/test.csv', index_col='PassengerId')
df_submit = pd.read_csv('./datasets/gender_submission.csv')

In [26]:
df_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [27]:
df_test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


## Prepare Data

In [28]:
def prep_data(df):
    df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

    df[['Age']] = df[['Age']].fillna(value=df[['Age']].mean())
    df[['Fare']] = df[['Fare']].fillna(value=df[['Fare']].mean())
    df[['Embarked']] = df[['Embarked']].fillna(value=df['Embarked'].value_counts().idxmax())

    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

    enbarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(enbarked_one_hot)

    return df

In [29]:
df_train = prep_data(df_train)
df_test = prep_data(df_test)

In [30]:
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,0,3,0,22.000000,1,0,7.2500,False,False,True
2,1,1,1,38.000000,1,0,71.2833,True,False,False
3,1,3,1,26.000000,0,0,7.9250,False,False,True
4,1,1,1,35.000000,1,0,53.1000,False,False,True
5,0,3,0,35.000000,0,0,8.0500,False,False,True
...,...,...,...,...,...,...,...,...,...,...
887,0,2,0,27.000000,0,0,13.0000,False,False,True
888,1,1,1,19.000000,0,0,30.0000,False,False,True
889,0,3,1,29.699118,1,2,23.4500,False,False,True


In [31]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,0,34.50000,0,0,7.8292,False,True,False
893,3,1,47.00000,1,0,7.0000,False,False,True
894,2,0,62.00000,0,0,9.6875,False,True,False
895,3,0,27.00000,0,0,8.6625,False,False,True
896,3,1,22.00000,1,1,12.2875,False,False,True
...,...,...,...,...,...,...,...,...,...
1305,3,0,30.27259,0,0,8.0500,False,False,True
1306,1,1,39.00000,0,0,108.9000,True,False,False
1307,3,0,38.50000,0,0,7.2500,False,False,True


X y

In [51]:
X = df_train.drop(['Survived'], axis=1).values.astype(float)
y = df_train['Survived'].values.reshape((-1,1))

## Scale X

In [52]:
scale = StandardScaler()
X = scale.fit_transform(X)

train, test

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# ML

# Catboost

# Keras

In [74]:
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

In [75]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
history = model.fit(X, y, epochs=1000, batch_size=16, workers=-1)

Epoch 1/1000
56/56 [==============================] - 1s 2ms/step - loss: 0.6995 - accuracy: 0.3838
Epoch 2/1000
56/56 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.3838
Epoch 3/1000
56/56 [==============================] - 0s 2ms/step - loss: 0.6866 - accuracy: 0.3838
Epoch 4/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6805 - accuracy: 0.3838
Epoch 5/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6774 - accuracy: 0.3838
Epoch 6/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6754 - accuracy: 0.3838
Epoch 7/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6673 - accuracy: 0.3838
Epoch 8/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6699 - accuracy: 0.3838
Epoch 9/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6695 - accuracy: 0.3838
Epoch 10/1000
56/56 [==============================] - 0s 1ms/step - loss: 0.6654 - accuracy: 0.3838

# Ans

In [ ]:
# df_submit['Survived'] = list(map(int, y_pred))
df_submit.to_csv('submission.csv', index=False)